# Basic plotting using Matplotlib and Numpy Data

Before we get started, you may choose to run this notebook on LEAP-Pangeo hub or Binder!

<a href="https://leap.2i2c.cloud/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fm2lines%2Fdata-gallery&urlpath=lab%2Ftree%2Fdata-gallery%2Fsrc%2Fnotebooks%2Focean_surface_boundary.ipynb&branch=main"><img src="https://custom-icon-badges.demolab.com/badge/LEAP-Launch%20%F0%9F%9A%80-blue?style=for-the-badge&logo=leap-globe" style="height:30px;"></a>

<a href="https://mybinder.org/v2/gh/m2lines/data-gallery/main?labpath=src%2Fnotebooks%2Focean_surface_boundary.ipynb"><img src="https://custom-icon-badges.demolab.com/badge/Binder-Launch%20%F0%9F%9A%80-blue?style=for-the-badge&logo=leap-globe" style="height:28px;"></a>

This notebook is built on top of `Parameterizing Vertical Mixing Coefficients in the Ocean Surface Boundary Layer Using Neural Networks` by Dr. Akash Sane and [Basic Plotting with MOM6](MOM6_basic_plotting.ipynb).

## Introduction



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
time = np.loadtxt("OHC_time.txt")  # time , days since reference time
K = 273  # this value does not matter because we are only checking OHC anomaly w.r.t 1958
th_NN = np.loadtxt("OHC_th_NN.txt") + K  # ocean mean temperature ePBL_NN
th_ML1 = np.loadtxt("OHC_th_ML1.txt") + K  # ocean mean temperature ePBL \gamma = 1
th_ML3 = np.loadtxt("OHC_th_ML3.txt") + K  # ocean mean temperature ePBL \gamma = 1
masso_NN = np.loadtxt("OHC_masso_NN.txt")  # Integral_vol (rho*vol)  ePBL_NN
masso_ML1 = np.loadtxt("OHC_masso_ML1.txt")  # Integral_vol (rho*vol)  ePBL \gamma = 1
masso_ML3 = np.loadtxt("OHC_masso_ML3.txt")  # Integral_vol (rho*vol)  ePBL \gamma = 3

# ocean heat content

ohc_MLE1 = masso_ML1 * th_ML1 * 3850  # Joules
ohc_NN = masso_NN * th_NN * 3850  # - ohc_MLE1[0]  # Joules
ohc_MLE3 = masso_ML3 * th_ML3 * 3850  # - ohc_MLE1[0] # Joules


i = 35  # index of year 1993
# Rate of Warming:\
Area = 361255840000000.0  # Area of ocean obtained from MOM6 OM4 grid. Units: m^2
dt = (
    (time[i:]) * 24 * 60 * 60
)  # time array in seconds between 2003 and 2017. Units: seconds


def get_ohc(ohc, Area, dt):
    d_ohc = (ohc[i:]) / Area
    p = np.polyfit(dt, d_ohc, 1)

    p1 = np.polyfit(dt, ohc[i:], 1)
    linear = np.polyval(p1, dt)

    return ohc, p, linear


## ePBL_NN
ohc_NN, p_nn, linear_NN = get_ohc(ohc_NN, Area, dt)

ohc_MLE1, p_mle1, linear_MLE1 = get_ohc(ohc_MLE1, Area, dt)

ohc_MLE3, p_mle3, linear_MLE3 = get_ohc(ohc_MLE3, Area, dt)

In [ ]:
### del this section:

ii = 0
plt.rcParams["lines.linewidth"] = 1
plt.rcParams["font.size"] = 12
plt.figure(figsize=(10, 5))
plt.plot(
    np.array(time) / 365 + 1958,
    (ohc_NN * 1e-21) - (ohc_NN * 1e-21)[ii],
    "r-",
    label="ePBL_NN",
)
plt.plot(
    np.array(time) / 365 + 1958,
    (ohc_MLE1 * 1e-21) - (ohc_MLE1 * 1e-21)[ii],
    "b-.",
    label=r"ePBL ($\gamma$=1)",
)
plt.plot(
    np.array(time) / 365 + 1958,
    (ohc_MLE3 * 1e-21) - (ohc_MLE3 * 1e-21)[ii],
    "g:",
    label=r"ePBL ($\gamma$=3)",
)

plt.plot(
    np.array(time[i:]) / 365 + 1958,
    (linear_NN * 1e-21) - (ohc_NN * 1e-21)[ii],
    color="k",
)
plt.plot(
    np.array(time[i:]) / 365 + 1958,
    (linear_MLE1 * 1e-21) - (ohc_MLE1 * 1e-21)[ii],
    color="k",
)
plt.plot(
    np.array(time[i:]) / 365 + 1958,
    (linear_MLE3 * 1e-21) - (ohc_MLE3 * 1e-21)[ii],
    color="k",
)

plt.annotate(
    str(np.around(p_mle3[0], 2)) + " W/m$^2$", (0.7, 0.2), xycoords="figure fraction"
)
plt.annotate(
    str(np.around(p_mle1[0], 2)) + " W/m$^2$", (0.7, 0.43), xycoords="figure fraction"
)
plt.annotate(
    str(np.around(p_nn[0], 2)) + " W/m$^2$", (0.65, 0.7), xycoords="figure fraction"
)

plt.xlabel("Years")
plt.ylabel("Ocean Heat Content, ZJ")
plt.legend()
# plt.show()
plt.savefig("ohc.pdf", format="pdf")